In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torch import distributions
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

True

UPload

In [3]:
trials = 1000
url = '../data/Haar4qubitsTrials'+str(trials)+'BIGsics.npy'
#url ='/content/drive/MyDrive/data4Ent/5qubitsmulti/5qubitsTrials'+str(trials)+'multinomialSics.npy'
#url = '/content/drive/MyDrive/data4Ent/MtoMdata/qutrit'+str(trials)+'RandomSic.npy'
#url2 = '/content/drive/MyDrive/data4Ent/q32/gltest_d32_SIC'+str(trials)+'.npy'

tomo = np.load(url, allow_pickle=True)


In [4]:
tomo.shape

(15000, 512)

In [5]:
tomo_train, tomo_temp = train_test_split(tomo, test_size=0.33, random_state=1)
tomo_valid,tomo_test = train_test_split(tomo_temp, test_size=0.7,random_state=1)


import random
valid = tomo_valid
train = tomo_train
test = tomo_test[:3000]

random.shuffle(train)
random.shuffle(valid)
random.shuffle(test)

train.shape, valid.shape, test.shape

((10050, 512), (1485, 512), (3000, 512))

##### Dataloader

In [6]:
class StatesDataset(Dataset):

    def __init__(self, x):
        super().__init__()
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index]

train_dataset = StatesDataset(train)
evaluation_dataset = StatesDataset(valid)

test_dataset = StatesDataset(test)

# Model and seed fixing

In [7]:

torch.cuda.empty_cache
del(Netbeta)


NameError: name 'Netbeta' is not defined

In [8]:

class Netbeta(nn.Module):
    def __init__(self, state_local_d, num_low_triang, out_channel, nheads, dim_ff ):
        super().__init__()

        kernel = 21
        kernel2 = 21
        

        padd = 10 #modello scon convtranspose era a 24, kernel1 = 49, all strid = 1
        strid1 = 1
        strid2 = 1
        model_dim = int((state_local_d + num_low_triang*2 - kernel + 2*padd)/strid1)+1


        #ENCODING
        self.conv1 = nn.Conv1d(1, out_channel, kernel_size = kernel, padding = padd,stride = strid1 )
        self.conv2 = nn.Conv1d(out_channel, 1, kernel_size = kernel2, padding = padd, stride = strid2 )


        #RECONSTRUCTION

        self.enc_transf = nn.TransformerEncoderLayer(d_model = model_dim, nhead = nheads, dim_feedforward = dim_ff, batch_first = True, norm_first=True)
        self.relu = torch.nn.ReLU()
        self.enc_stack = nn.TransformerEncoder(self.enc_transf, num_layers=2)
        self.T = torch.nn.Tanh()
        self.G = torch.nn.GELU()


    def forward(self, x):

        #ENCODING

        x = self.G(self.conv1(x)) #this for pauli's
        #x= F.selu(self.conv1(x))

        #DECODING

        x= self.G(self.enc_stack(x))

        x = self.T(self.conv2(x))
        x = torch.flatten(x,1) 

        return x



# Training

leading parameters

In [9]:

local_dim = 16
dim = 1
torch.manual_seed(13)

network parameters

In [10]:
num_killed_inputs = 0

input_dim = local_dim**(2*dim)
batch_size = 500

statedim = local_dim**dim
lowerval = (statedim**2 -statedim)/2
print(lowerval)

device = "cuda"
learning_rate = 0.0001
num_epochs = 1000



netb = Netbeta(int(local_dim) , int(lowerval), 2,1,20)
themodelbeta  = netb.to(device)
themodelbeta = themodelbeta.double()


def normalization_loss(inp):

  batchsize = inp.shape[0]

  #return torch.abs(torch.sum(torch.norm(single_vector, p ='fro', dim=1))/batchsize -1 )
  return  torch.mean(torch.norm(inp, p ='fro', dim=1))/batchsize

def diceCost(inp,target):
    
    a = torch.sum(torch.mul(inp,target),1)
    b = torch.sum(torch.mul(inp,inp),1)
    c = torch.sum(torch.mul(target,target),1)

    return 1-  torch.mean(torch.div(2*a, (b+c) ))

  

120.0


In [11]:

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    shuffle=True,
    pin_memory=True)

valid_dataloader = torch.utils.data.DataLoader(
    evaluation_dataset, batch_size=batch_size,
    shuffle=True,
    pin_memory=True)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size,
    shuffle=True,
    pin_memory=True)


In [12]:

rec_loss = torch.nn.MSELoss()
#optimizer = optim.RMSprop(themodelbeta.parameters(), lr=learning_rate)
optimizer = optim.Rprop(themodelbeta.parameters(), lr = learning_rate, step_sizes = (1e-6,1e-4))


val_hist = []
train_hist = []


for epoch in range(num_epochs):
    train_loss = 0
    for inputs in train_dataloader:

        themodelbeta.train()

        inputs = inputs.to(device)
        optimizer.zero_grad()


        #INPUTS RESHAPING

        newin = torch.reshape(inputs[:,:local_dim**(2*dim)  ], (inputs.shape[0],1,local_dim**(2*dim )) )
        target = inputs[:,local_dim**2:]
        out = themodelbeta(newin)

        loss = rec_loss(out, target) + normalization_loss(out)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    valid_loss = 0
    for inputs in valid_dataloader:
        with torch.no_grad():
            themodelbeta.train()

            inputs = inputs.to(device)
            optimizer.zero_grad()
            target = inputs[:,local_dim**2:]


            #INPUTS RESHAPING

            newin = torch.reshape(inputs[:,:local_dim**(2*dim)  ], (inputs.shape[0],1,local_dim**(2*dim )) )
            out = themodelbeta(newin)

            loss = rec_loss(out, target) + normalization_loss(out)

            valid_loss += loss.item()


    print(epoch, train_loss/len(train_dataloader), valid_loss/len(valid_dataloader))
    val_hist.append(valid_loss/len(valid_dataloader))
    train_hist.append(train_loss/len(train_dataloader))

0 0.03997004126188461 0.02272932561214167
1 0.019632285835484476 0.01577926550040681
2 0.015495198755581524 0.013103984744590555
3 0.01290265391819119 0.010961174929890455
4 0.010759318706236403 0.009120204486137748
5 0.009039740531623826 0.007691092476146146
6 0.0076449431707188665 0.006591620811038724
7 0.006572356156672209 0.005797687886818908
8 0.006000161148757798 0.0054769521735628926
9 0.005757746691995093 0.005313597955102775
10 0.005579542521687646 0.005186182408474034
11 0.005461057993701282 0.005097241518153459
12 0.005345233439691535 0.005018414987857803
13 0.005266303757406117 0.004929696935589048
14 0.005154459122641665 0.004859329666246778
15 0.005104061612607323 0.004793306659415464
16 0.0050162411723055815 0.0047348600841602825
17 0.004954103001920383 0.0046834661777746525
18 0.0048826854363047635 0.00462569526425708
19 0.004832673555990856 0.004589088379372722
20 0.004784739823254928 0.004538742895769609
21 0.004721745573048903 0.00449279899485423
22 0.004667140834505

KeyboardInterrupt: 

# Test. HS and fidelity reconstruction

## matrix reconstruction functions

In [13]:
def return_matrix_elements(stuff,d,local_dim):

	diag_len = local_dim**d
	diagel = stuff[local_dim**(2*d) : local_dim**(2*d)  + diag_len  ]
	offd = stuff[ local_dim**(2*d)  + diag_len :  ]
	return diagel, offd

In [14]:
def rebuild_last(diags,offd,ind):
    '''''
    input.
    diags : diags elements
    offds : offdiagonal elements
    rhoshape : just the dm dimension, e.g. number of rows
    localDim : the number of level of each particle
    numParticles : the total number of particles for tensor product

    output.
    m : reconstructed cholesky decoposition matrix
    '''''
    d = len(diags)
    eye = np.eye(d,d)
    mat = np.zeros((d,d),dtype = complex)

    offvalues = [ (a+1j*b) for (a,b) in zip(offd[:int(len(offd)/2 ) ], offd[int(len(offd)/2):])  ]
    mat[ind[0], ind[1]] = offvalues

    return mat + eye*diags

## test stage

In [15]:
#just generate ones the index for the recosntruction using a dull matrix

import qutip as qt
def fid(a,b):
  fid = qt.fidelity(qt.Qobj(a), qt.Qobj(b))
  return torch.tensor(fid)


#parameters
qfid =0
j=0
hs = []
fids =[]

# generate support diagonal matrix for reconstruction
e = np.eye(local_dim**(dim),local_dim**(dim))
print(e.shape)

#indeces for the reconstruction function
ind = np.tril_indices_from(e,k=-1)


(16, 16)


/home/amacarone/miniconda3/envs/cudaml2/lib/python3.11/site-packages/qutip/__init__.py:96: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


In [16]:
def bruteForce(dm):

	eis, eigvecs = np.linalg.eig(dm)

	eis[eis <0 ] = 0.00001
	eis = [ el.real for el in eis]
	eis = np.array(eis)/sum(eis)

	cleanRho = eigvecs@ np.diag(eis) @ eigvecs.T.conj()

	return cleanRho

In [17]:
file_ = []
originals = []
with torch.no_grad():
  for inputs in test_dataloader:

    inputs = inputs.to(device)

    #out_d is the diagonal, out_tr the triangluar values, real and complex altogether
    newin = torch.reshape(inputs[:,:local_dim**(2*dim) ], (inputs.shape[0],1,local_dim**2) )

    out = themodelbeta(newin)

    for i in range(out.shape[0]):

      #reconstructing cholesky from dataset input array. NO MORE NEEDED
      diagel, offd = return_matrix_elements( inputs[i].cpu().numpy(), dim, local_dim)

      original_chol = rebuild_last(diagel, offd , ind)

      #reconstructing cholesky from neural network outputs

      nn_out = out[i].cpu().numpy()
      nn_diag = nn_out[:local_dim]
      nn_offd = nn_out[local_dim: local_dim**2]

      nn_chol = rebuild_last(nn_diag, nn_offd, ind)

      cholo=nn_chol@ nn_chol.conj().T
      norm = np.trace(cholo)


      #fidelity between reconstructed and originals

      fids.append(fid(original_chol@ original_chol.conj().T,cholo/norm) )
      hs.append(qt.hilbert_dist(qt.Qobj(cholo/norm), qt.Qobj(original_chol@ original_chol.conj().T)))

      file_.append(cholo/norm)
      originals.append(original_chol@ original_chol.conj().T)

print(f"averaged quantum fidelity {np.mean(fids): .4f} ")#dont forget the j to divide by

print(np.std(fids))
print(f"hilbert schmidt distance average {np.mean(hs): .4f}")
print(f"hilbert schmidt distance std {np.std(hs): .4f}")


averaged quantum fidelity  0.6113 
0.13399513
hilbert schmidt distance average  0.7281
hilbert schmidt distance std  0.2522


## file saving

In [40]:
torch.save(themodelbeta, 'modelsHAAR4qubits'+str(trials)+'-FID97Pauli.pth' )

## model delating

## third party data

In [ ]:
test_gl = gl

gl_dataset = StatesDataset(test_gl)
gl_dataloader = torch.utils.data.DataLoader(
    gl_dataset, batch_size = batch_size, shuffle =False,
    pin_memory = True)

In [ ]:
glFile = []

name = 'q32TestSet_SICtrials_'+str(trials)+'v2.npy'

with torch.no_grad():
  for inputs in gl_dataloader:

    inputs = inputs.to(device)

    #out_d is the diagonal, out_tr the triangluar values, real and complex altogether
    newin = torch.reshape(inputs[:,: ], (inputs.shape[0],1,local_dim**2) )

    out_d, out_tr = themodelbeta(newin)

    for i in range(out_d.shape[0]):

      #reconstructing cholesky from neural network outputs

      nn_diag = out_d[i].cpu().numpy()
      nn_offd = out_tr[i,:int(out_tr.shape[1])].cpu().numpy()

      nn_chol = rebuild_last(nn_diag, nn_offd, ind)

      cholo=nn_chol@ nn_chol.conj().T
      norm = np.trace(cholo)

      glFile.append(cholo/norm)

  np.save(name, np.array(glFile))

In [ ]:
PATH = '/content/drive/MyDrive/models5qubitsBest/500Kfid812multinomial.pth'

m = torch.load(PATH)
m.eval()